# Bases de données SQLite

## Présentation

**SQL :** *Structured Query Language*

Il s’agit d’un langage de requête structurée pour interroger des bases de données. Il repose sur des verbes (`CREATE`, `SELECT`, `INSERT`, `DELETE`…) qui effectuent des actions sur des modèles de données. Il est généralement mis en œuvre par des programmes via des interfaces pour se connecter à des systèmes de gestion de bases de données relationnelles (SGBDR).

**SQLite :** bibliothèque qui manipule une base de données légère et indépendante de tout SGBDR. SQLite mobilise l’essentiel des commandes de SQL sans être pour autant exhaustif.

En Python, le module `sqlite3` se charge de fournir une interface avec la bibliothèque :

In [ ]:
import sqlite3

Toute transaction doit se rattacher à une connexion et être manipulée par un objet de type `Cursor` :

In [ ]:
connexion = sqlite3.connect("./data/covid.db")

Une fois la connexion à la base de données établie, instancier un objet `Cursor` qui sera chargé de manipuler la base de données :

In [ ]:
c = connexion.cursor()

La méthode `execute()` de tout objet `Cursor` lancera les requêtes SQL en renvoyant un itérateur :

In [ ]:
sql = "SELECT department FROM departments LIMIT 5"
departments = c.execute(sql)

Une boucle `for` traditionnelle suffit pour parcourir cet itérateur :

In [ ]:
for department in departments:
    print(department)

Attention, un itérateur `Cursor` n’est valable qu’une seule fois !

In [ ]:
for department in departments:
    print(department)

Pour sauvegarder le résultat d’une requête, utiliser les méthodes `fetchone()` ou `fetchall()` :

In [ ]:
department = c.execute(sql).fetchone()
departments = c.execute(sql).fetchall()

**Remarque :** le résultat d’une requête prend toujours la forme d’un tuple.

## Structure de la base de données

Avant d’effectuer des requêtes sur une base de données, il est utile d’en connaître la structure, comme le nom des tables et la description de leurs champs.

### Nom des tables

In [ ]:
sql = """SELECT name
FROM sqlite_master
WHERE type ='table'
    AND name NOT LIKE 'sqlite_%';"""
for row in c.execute(sql):
    print(row)

### Lister les champs d’une table

In [ ]:
sql = """SELECT sql
FROM sqlite_master 
WHERE name = 'departments';"""
for row in c.execute(sql):
    print(row)

### Obtenir le détail des champs

In [ ]:
sql = "PRAGMA table_info('departments')"
for row in c.execute(sql):
    print(row)

Les informations sont, dans l’ordre : l’identifiant unique du champ, son nom, son type, s’il s’agit d’un champ facultatif ou non (0 : facultatif ; 1 : obligatoire), sa valeur par défaut et, enfin, s’il s’agit d’une clé primaire ou non.

## Gérer une base de données

### Création de la base

La méthode `connect()` est suffisante pour créer un fichier *.db* :

In [ ]:
connexion = sqlite3.connect('./data/mauritian.bd')

Toutes les opérations qui suivront devront se rattacher à un objet `Cursor` :

In [ ]:
c = connexion.cursor()

### Créer une table

In [ ]:
sql = "CREATE TABLE lexicon (word, etymon, phono)"
c.execute(sql)

### Insérer un enregistrement

In [ ]:
sql = "INSERT INTO lexicon VALUES (?, ?, ?)"
c.execute(sql, ('abandone', 'abandonner', 'abãdone'))

### Insérer plusieurs enregistrements

Cette fois-ci, il faut faire appel à la méthode `executemany()` à laquelle on passe une liste de tuples en plus de la requête SQL :

In [ ]:
sql = "INSERT INTO lexicon VALUES (?, ?, ?)"
records = [
    ('abdike', 'abdiquer', 'abdike'),
    ('abwaye', 'aboyer', 'abwaje'),
    ('aktialize', 'actualiser', 'aktualize')
]
c.executemany(sql, records)

### Supprimer des enregistrements

La requête `DELETE FROM {table}` permet de supprimer des enregistrements. Sans condition, elle supprime tous les enregistrements d’une table :

In [ ]:
# sql = "DELETE FROM lexicon" would delete all the records.
sql = "DELETE FROM lexicon WHERE word=:word"
c.execute(sql, {'word': 'abdike'})

Remarquez la syntaxe de transmission des paramètres : lors d’un enregistrement, on utilise des points d’interrogation (*qmark style*) et pour une sélection plutôt des paramètres nommés (*named style*).

### Mettre à jour des enregistrements

La transcription phonologique du mot *aktialize* est erronnée. On a enregistré *aktualize* au lieu de *aktjalize*. Corrigeons :

In [ ]:
sql = "UPDATE lexicon SET phono=:phono WHERE word=:word"
c.execute(sql, {'word': 'aktialize', 'phono': 'aktjalize'})

### Supprimer une table

In [ ]:
c.execute("DROP TABLE lexicon")

### Sauvegarder les opérations sur la base de données

Toutes les transactions (création, modification, suppression) doivent obligatoirement être sauvegardées avant de rompre la connexion, faute de quoi, tout le travail aura été perdu !

La sauvegarde s’effectue avec la méthode `commit()` et la fermeture de la connexion avec `close()`.

In [ ]:
import sqlite3
connexion = sqlite3.connect("./data/mauritian.bd")
c = connexion.cursor()
c.execute("CREATE TABLE lexicon (word, etymon, phono)")
c.execute("INSERT INTO lexicon VALUES (?, ?, ?)", ('abandone', 'abandonner', 'abãdone'))
connexion.commit()
connexion.close()

## Indexation et performance

Les mécanismes d’indexation constituent l’avantage majeur d’une base de données SQLite par rapport à un fichier plat en ce qu’elles améliorent les performances de chargement et de recherche d’information.

### Définir une clé primaire

Une clé primaire est un outil très puissant pour à la fois indexer une colonne particulière mais aussi assurer l’intégrité des données. Une colonne disposant d’une clé primaire n’acceptera pas de valeur nulle et ne pourra pas non plus contenir de valeurs en double. Soulignons aussi qu’une table ne peut accueillir qu’une seule clé primaire, mais cette clé peut être posée sur plusieurs colonnes.

In [ ]:
# While the creation of the table
sql = "CREATE TABLE lexicon (word PRIMARY KEY, etymon, phono, tag)"
# When the table already exists
sql = "ALTER TABLE lexicon ADD PRIMARY KEY (word)"
# A composite primary key
sql = "ALTER TABLE lexicon ADD CONSTRAINT pk_word_tag PRIMARY KEY (word, tag)"

Et pour supprimer une clé primaire :

In [ ]:
sql = "ALTER TABLE lexicon DROP PRIMARY KEY"

### Gérer des index

À l’instar d’un index à la fin d’un livre, les index d’une base de données sont comme des pointeurs vers les données. Ils améliorent nettement la rapidité d’exécution d’une instruction `SELECT` ainsi que de toutes les clauses `WHERE`.

#### Créer un index

In [ ]:
sql = "CREATE INDEX pos_tag ON lexicon (tag)"

La syntaxe de base peut se compléter avec le mot-clé `UNIQUE` pour vérifier qu’aucune valeur en double ne sera ajoutée dans la colonne :

In [ ]:
sql = "CREATE UNIQUE INDEX word_index ON lexicon (word)"

La commande `CREATE INDEX` accepte également une clause `IF NOT EXISTS` pour éviter le retour d’une erreur si l’index était déjà présent :

In [ ]:
sql = "CREATE INDEX IF NOT EXISTS pos_tag ON lexicon (tag)"

Un index peut également concerner plusieurs colonnes. On parle alors d’index composites :

In [ ]:
sql = "CREATE INDEX word_tag ON lexicon (word, tag)"

#### Supprimer un index

In [ ]:
sql = "DROP INDEX word"

#### Lister les index d’une table

Retrouver tous les index d’une table ne se fait malheureusement pas simplement avec une commande unique. La méthode consiste à interroger la table maîtresse `SQLite_master` qui pilote la bases de données et à lister l’ensemble des déclarations SQL :

In [ ]:
connexion = sqlite3.connect("./data/mauritian.bd")
c = connexion.cursor()
c.execute("SELECT sql from sqlite_master")
statements = c.fetchall()
for statement in statements:
    print(statement)

## Techniques courantes

### Définir une valeur par défaut

In [ ]:
sql = "CREATE TABLE lexicon (word, etymon, phono, tag DEFAULT 'V')"

### Attribuer un type de données

In [ ]:
sql = "CREATE TABLE lexicon (word TEXT, etymon TEXT, phono TEXT, tag TEXT)"

### Créer une table si nécessaire

In [ ]:
sql = "CREATE TABLE IF NOT EXISTS lexicon (word, etymon, phono, tag)"

### Trier

In [ ]:
sql = "SELECT word, phono FROM lexicon ORDER BY word"